# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem <a name="introduction"></a>

## Data<a name="data"></a>

## Methodology<a name="methodology"></a>

## Analysis<a name="analysis"></a>

## Results<a name="results"></a>

## Conclusion<a name="conclusion"></a>